In [2]:
import requests
import pandas as pd

# Date to test
date = "2015-01-01"
url = f"https://sinav30.conagua.gob.mx:8080/PresasPG/presas/reporte/{date}"

# Get and parse response
response = requests.get(url)
response.raise_for_status()
data = response.json()

# Handle different response formats
if isinstance(data, list):
    # Some older dates may return just the facility-level list
    df_facility = pd.DataFrame(data)
    df_state = pd.DataFrame()  # empty
elif isinstance(data, dict):
    df_facility = pd.DataFrame(data.get("presasMexico", []))
    df_state = pd.DataFrame(data.get("presasEntidadFederativa", []))
else:
    raise ValueError("Unexpected data format")

# Show results
print("=== Facility Report ===")
print(df_facility.head())

print("\n=== State Report ===")
print(df_state.head() if not df_state.empty else "[No state-level data for this date]")


=== Facility Report ===
   idmonitoreodiario fechamonitoreo clavesih  \
0            1486282     2015-01-01    CUQJL   
1            1486288     2015-01-01    EGCJL   
2            1486289     2015-01-01    EGLGR   
3            1486290     2015-01-01    ELCZC   
4            1486291     2015-01-01    ELOSL   

                      nombreoficial      nombrecomun           estado  \
0                      Cuquío, Jal.     Los Gigantes          Jalisco   
1  Ing. Elías González Chávez, Jal.  Puente Calderón          Jalisco   
2                    El Gallo, Gro.         El Gallo         Guerrero   
3                 El Cazadero, Zac.      El Cazadero        Zacatecas   
4           Cañada del Lobo, S.L.P.                   San Luis Potosí   

          nommunicipio                    regioncna    latitud    longitud  \
0               Cuquío      Lerma-Santiago-Pacífico  20.948056 -103.039444   
1          Zapotlanejo      Lerma-Santiago-Pacífico  20.681389 -102.976111   
2  Cutzamala d

In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta

start_date = datetime(2015, 1, 1)
end_date = datetime.now()

facility_records = []
state_records = []

current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime("%Y-%m-%d")
    url = f"https://sinav30.conagua.gob.mx:8080/PresasPG/presas/reporte/{date_str}"

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()

        if isinstance(data, list):
            for item in data:
                item['fecha_reporte'] = date_str
            facility_records.extend(data)
        elif isinstance(data, dict):
            for item in data.get("presasMexico", []):
                item['fecha_reporte'] = date_str
            for item in data.get("presasEntidadFederativa", []):
                item['fecha_reporte'] = date_str
            facility_records.extend(data.get("presasMexico", []))
            state_records.extend(data.get("presasEntidadFederativa", []))

        print(f"✅ {date_str}")

    except Exception as e:
        print(f"❌ {date_str}: {e}")

    current_date += timedelta(days=1)

# Save to CSV or explore in pandas
df_facility = pd.DataFrame(facility_records)
df_state = pd.DataFrame(state_records)

df_facility.to_csv("conagua_facility_all.csv", index=False, encoding="utf-8-sig")
df_state.to_csv("conagua_state_all.csv", index=False, encoding="utf-8-sig")

print("✅ Done. CSVs saved.")


✅ 2015-01-01
✅ 2015-01-02
✅ 2015-01-03
✅ 2015-01-04
✅ 2015-01-05
✅ 2015-01-06
✅ 2015-01-07
✅ 2015-01-08
✅ 2015-01-09
✅ 2015-01-10


KeyboardInterrupt: 